In [ ]:
#Initialize constants
#PUT THE NUMBER OF POINTS IN YOUR SPACE HERE (must be at least 3)
spaceSize = 8 #points

dimensions = (spaceSize*(spaceSize-1))/2
print("Space Size: " + str(spaceSize) + " points")
print("Dimension of cone: " + str(dimensions))

In [ ]:
#Make a matrix that indexes the edges between points of space
edgeIndex = [[-1 for j in range(spaceSize)] for i in range(spaceSize)]
currentIndex = 0
for i in range(spaceSize):
    for j in range(i+1,spaceSize):
        edgeIndex[i][j] = currentIndex
        currentIndex += 1

print("Edge index: " + str(edgeIndex))

def verticesToEdgeIndex(v,w):
    return edgeIndex[min(v,w)][max(v,w)]

In [ ]:
#Construct vectors representing triangle inequalities.
#Each vector is normal to the boudary of the
#halfspace satisfying its corresponding inequality.
#We will use these to construct the cone
inequalities = []
zeroVector = []
for i in range(0,dimensions):
    zeroVector.append(0)

for i in range(0,spaceSize):
    for j in range(i+1,spaceSize):
        for k in range(j+1,spaceSize):
            edge1=verticesToEdgeIndex(i,j)
            edge2=verticesToEdgeIndex(j,k)
            edge3=verticesToEdgeIndex(i,k)
            
            newVector1 = zeroVector[:]
            newVector2 = zeroVector[:]
            newVector3 = zeroVector[:]
            
            newVector1[edge1]=1
            newVector1[edge2]=1
            newVector1[edge3]=-1
            
            newVector2[edge1]=1
            newVector2[edge2]=-1
            newVector2[edge3]=1
            
            newVector3[edge1]=-1
            newVector3[edge2]=1
            newVector3[edge3]=1
            
            inequalities.append(newVector1)
            inequalities.append(newVector2)
            inequalities.append(newVector3)
print("Triangle inequalities: " + str(len(inequalities)))

In [ ]:
#Construct cone as dual of cone generated by inequality vectors
#This will be a minimal generating set for the cone, so we
#can set check=False. This saves a lot of computation time
dualCone = Cone(inequalities, check=False)

In [ ]:
#The irreducible pseudometrics are the vectors that are normal to the
#facets of the dual cone.
#This step takes a long time

print("Computing normal vectors to facets...")
irreducibles = dualCone.facet_normals()
print("There are " + str(len(irreducibles)) +
      " irreducible pseudometrics on " + str(spaceSize) +
      " points, not controlling for symmetry.")

In [ ]:
#Turn irreducible pseudometrics into weighted graph representations
#and filter out non-metrics
def pseudoToGraph(pseudometric):
    graph = graphs.CompleteGraph(spaceSize)
    graph.weighted(new=True)
    for i in range(spaceSize):
        breakout = False
        for j in range(i+1,spaceSize):
            edge = verticesToEdgeIndex(i,j)
            weight = pseudometric[edge]
            graph.set_edge_label(i,j,weight)
            if weight == 0:
                return None
    return graph

weightedGraphsAndPseudos = []
for pseudometric in irreducibles:
    graph = pseudoToGraph(pseudometric)
    if graph != None:
        weightedGraphsAndPseudos.append((graph,pseudometric))
print("Graphs created: " + str(len(weightedGraphsAndPseudos)))

In [ ]:
#Filter graphs for isomorphic copies
filteredWeightedGraphsAndPseudos = []
for currentPair in weightedGraphsAndPseudos:
    currentGraph = currentPair[0]
    isNewGraph = True
    for clearedPair in filteredWeightedGraphsAndPseudos:
        clearedGraph = clearedPair[0]
        if currentGraph.is_isomorphic(clearedGraph, edge_labels = true):
            isNewGraph = False
            break
    if isNewGraph:
        filteredWeightedGraphsAndPseudos.append(currentPair)
print("There are " + str(len(filteredWeightedGraphsAndPseudos)) + " irreducible metrics on " + str(spaceSize) + " points, controlling for symmetry")

In [ ]:
def displayGraph(graph):
    graph.plot(edge_labels=true, layout = "circular").show()

In [ ]:
#Check which pseudometrics are generated by unweighted graphs
#and print results


def removeNonunitEdges(graph):
    newGraph = graph.copy()
    for edge in newGraph.edges():
        u,v,label = edge
        weight = int(label)
        if weight != 1:
            newGraph.delete_edge(u,v)
    return newGraph

def raiseAStink(pseudo,weightedGraph,unweightedGraph):
    print("Pseudometric does not appear to be generated by unweighted graph.")
    print("Pseudometric:" + str(pseudo))
    print("Weighted graph:")
    displayGraph(weightedGraph)

graphablePseudos = 0
ungraphablePseudos = 0
for weightedGraph,pseudo in filteredWeightedGraphsAndPseudos:
    unweightedGraph = removeNonunitEdges(weightedGraph)
    if not unweightedGraph.is_connected():
        ungraphablePseudos += 1
        raiseAStink(pseudo,weightedGraph,unweightedGraph)
        continue
    
    weightedGraphMatrix = weightedGraph.weighted_adjacency_matrix()
    unweightedGraphMatrix = unweightedGraph.distance_matrix()
    if(weightedGraphMatrix == unweightedGraphMatrix):
        graphablePseudos += 1
        print("Pseudometric " + str(pseudo))
        displayGraph(weightedGraph)
        print(" can be represented by unweighted graph")
        displayGraph(unweightedGraph)
    else:
        #Raise a stink
        ungraphablePseudos += 1
        raiseAStink(pseudo,weightedGraph,unweightedGraph)
        continue
print(str(graphablePseudos) + " of the irreducible metrics could be"
      + " represented by unweighted graphs.")
print(str(ungraphablePseudos) + " could not.")

In [ ]:
#Optionally, save list of pairs of irreducible pseudometrics and their
#graph representations
save(filteredWeightedGraphsAndPseudos,"exhaustiveSearchFinds/nOf" + str(spaceSize))

In [ ]:
#Optionally, load saved list
filteredWeightedGraphsAndPseudos = load("exhaustiveSearchFinds/nOf" + str(spaceSize))
print("Loaded " + str(len(filteredWeightedGraphsAndPseudos)) +
      " pseudometrics on " + str(spaceSize) +
      " points, and representative graphs")